# Interactive heatmap demo using `bokeh`

In [1]:
# must go first
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Reloads functions each time so you can edit a script 
# and not need to restart the kernel
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import IFrame

import warnings
warnings.filterwarnings('ignore')

Please make sure bokeh is installed (`pip install bokeh`). This notebook was developed using `bokeh==2.2.3` but it should be compatible with other versions. 

Must use the code below to get Bokeh working within the notebook:

In [2]:
import bokeh
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
import vishelper as vh

## Where to save figures

In [4]:
savepath_prefix = 'figures/'

## Create fake data

In [5]:
df = pd.DataFrame()
df['date'] = pd.date_range(start='1/1/2020', end='12/31/2020')

Create fake values for plotting

In [6]:
df['fake_data'] = np.random.randint(0, 50, len(df))

Make a number of days having nothing recorded (`fake_data=0`)

In [7]:
df.loc[df.sample(75).index, 'fake_data'] = 0

Make Wednesdays take much higher values

In [8]:
df.loc[df.date.dt.dayofweek==2, 'fake_data'] = np.random.randint(50, 100, len(df[df.date.dt.dayofweek==2]))

Make every 4 Fridays take higher values

In [9]:
every_four_weeks = (df.date.dt.dayofweek==4) & (df.date.dt.weekofyear % 4 == 0)
df.loc[every_four_weeks, 'fake_data'] = np.random.randint(75, 125, every_four_weeks.sum())

## Add other fake data for tool tips

In [10]:
df['fake_added_info'] = "I am added info"

## Create features for plotting 

Create features for day of week and week of year

In [11]:
df['dayofweek'] = df['date'].dt.dayofweek

In [12]:
df['weekofyear'] = df['date'].dt.weekofyear

Create labels

In [13]:
df["week_begins"] = df.apply(lambda x: x['date'] - pd.Timedelta("%i D" % x['dayofweek']), axis=1)
df['weekof'] = df.week_begins.dt.strftime('%B %d, %Y')

df['week_begins'] = df.week_begins.dt.strftime('%Y-%m-%d')
df['date_str'] = df.date.dt.strftime('%Y-%m-%d')

In [14]:
df.head()

,date,fake_data,fake_added_info,dayofweek,weekofyear,week_begins,weekof,date_str
0,2020-01-01,86,I am added info,2,1,2019-12-30,"December 30, 2019",2020-01-01
1,2020-01-02,14,I am added info,3,1,2019-12-30,"December 30, 2019",2020-01-02
2,2020-01-03,18,I am added info,4,1,2019-12-30,"December 30, 2019",2020-01-03
3,2020-01-04,0,I am added info,5,1,2019-12-30,"December 30, 2019",2020-01-04
4,2020-01-05,7,I am added info,6,1,2019-12-30,"December 30, 2019",2020-01-05


## Prepare y-axis labels

### Define y-axis labels

In [15]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

### Convert day of week to name of day 
Making the `dayofweek` align with the labels we are providing for the y-axis

In [16]:
df['dayofweek'] = df.dayofweek.apply(lambda x: days[x])

## Prepare x-axis labels

In [17]:
weeks = df.sort_values(by='week_begins').weekof.unique()

# Plotting

## Set up tooltips 

Tool tips can be provided by a list of tuples. Each tuple has the label for the value to be displayed and then the name of the column containing that value, beginning with an `@`: 

```python
tooltips = [('Label of 1st piece of information', '@column_a'),
            ('Label of 2nd piece of information', '@column_b')]
```

In [18]:
tooltips = [('Date', '@date_str'), ('Fake data value', '@fake_data'), 
           ('More useful info:', '@fake_added_info')]

## Create the plot

Must save at html, then can display in `IFrame`

In [19]:
savepath=os.path.join(savepath_prefix, 'fake-heatmap-0.html')

In [20]:
p = vh.interactive_heatmap(df,
                           save_path=savepath,
                           value_column='fake_data',
                           y_range=days,
                           ycolumn='dayofweek',
                           xcolumn='weekof',
                           xlabel='Week of',
                           x_range=weeks,
                           colorbar_format="%d things",
                           tooltips=tooltips,
                           title="# of things by day of week and week of year")

## Display the plot

In [21]:
IFrame(os.path.relpath(savepath), width=900, height=600)

## Location choices

Switch locations of objects: 

```python
x_axis_location="below",
toolbar_location='above',
```

In [22]:
savepath1=os.path.join(savepath_prefix, 'fake-heatmap-1.html')

In [23]:
p = vh.interactive_heatmap(df,
                           save_path=savepath1,
                           value_column='fake_data',
                           y_range=days,
                           ycolumn='dayofweek',
                           xcolumn='weekof',
                           xlabel='Week of',
                           x_range=weeks,
                           colorbar_format="%d things",
                           tooltips=tooltips,
                           x_axis_location="below",
                           toolbar_location='above',
                           title="# of things by day of week and week of year")

In [24]:
IFrame(os.path.relpath(savepath1), width=900, height=600)

## Change colorbar location

```python
y_axis_location = 'right'
colorbar_location = 'left'
```

In [25]:
savepath3=os.path.join(savepath_prefix, 'fake-heatmap-3.html')

In [26]:
p = vh.interactive_heatmap(df,
                           save_path=savepath3,
                           value_column='fake_data',
                           y_range=days,
                           ycolumn='dayofweek',
                           xcolumn='weekof',
                           x_range=weeks,
                           colorbar_format="%d things",
                           xlabel='Week of',
                           y_axis_location='right',
                           colorbar_place='left',
                           tooltips=tooltips,
                           title="# of things by day of week and week of year")

Not quite sure how to change orientation of colorbar ticks...

In [27]:
IFrame(os.path.relpath(savepath3), width=900, height=600)

## Horiztonal colorbar

The below parameters are recommended for creating a horizontal colorbar. 

```python
colorbar_orientation='horizontal',
colorbar_place='below',
min_border_right=80,
colorbar_label_standoff=5,              
```

In [28]:
savepath4=os.path.join(savepath_prefix, 'fake-heatmap-4.html')

In [29]:
p = vh.interactive_heatmap(df,
                           save_path=savepath4,
                           value_column='fake_data',
                           y_range=days,
                           ycolumn='dayofweek',
                           xcolumn='weekof',
                           xlabel='Week of',
                           x_range=weeks,
                           colorbar_format="%d things",
                           colorbar_orientation='horizontal',
                           colorbar_place='below',
                           min_border_right=80,
                           colorbar_label_standoff=5,
                           tooltips=tooltips,
                           plot_height=500,
                           title="# of things by day of week and week of year")

In [30]:
IFrame(os.path.relpath(savepath4), width=900, height=600)

# Appendix
## Watermark 
For full reproducibility of results, use exact data extraction as defined at top of notebook and ensure that the environment is exactly as follows: 

In [31]:
# ! pip install watermark
%load_ext watermark
%watermark -v -m --iversions -g

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.2.0

Compiler    : Clang 12.0.1 
OS          : Darwin
Release     : 21.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 10
Architecture: 64bit

Git hash: 2d18bae00e50ac83267cb38ce1d89451ac6692e1

vishelper : 0.1.2
bokeh     : 2.4.2
pandas    : 1.4.2
matplotlib: 3.5.1
json      : 2.0.9
numpy     : 1.22.3

